# Mount Google Drive

In [1]:
# from google.colab import data_table
# data_table.enable_dataframe_formatter()

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# from google.colab import drive
# drive.mount('/gdrive')

# Import Experiment Data

In [4]:
# %cd /gdrive/My Drive/F3/ #REPLACE YOUR GOOGLE DRIVE DIRECTORY HERE
# %ls

In [5]:
import os
import pandas as pd

In [6]:
from dotenv import load_dotenv
load_dotenv('../.env')

True

In [7]:
pd.set_option('display.max_columns', 100)

In [8]:
df = pd.read_csv('../F3_Dataset/Full Clean Dataset/F3_Consistency.csv')
print(df.shape)
df[:3]

(27667, 50)


,uuid,human_content,ai_content,model,num_completion_token,num_original_token,num_prompt_token,num_iagenerated_token,original_label,source_type,ai_generated_label,article_type,pre_post_GPT,dataset_source,Prompt_type,GPT_Entailment_Explanation,GPT_NLI_Label,FLAN_NLI_Label,BERTScore,BLEURT-20,aigenerated_content_cleaned,emoji_count,tokens_ai,tokens_human,diff_ai_to_human,diff_human_to_ai,Multicall_labels,AlignScore,human_TextBlob_polarity,human_TextBlob_subjectivity,ai_TextBlob_polarity,ai_TextBlob_subjectivity,human_embed,ai_embed,semantic_distance,edit_distance,jaccard_similarity,overlap_coefficient,combined_hue,Text-bison-NLI,PaLM_NLI,PaLM_categories,uuid.1,Llama2-Entailment,Llama_NLI,GPT_NLI_plus_ai_label,Llama_NLI_plus_ai_label,PaLM_NLI_plus_ai_label,Logical Consistency,factual_consistency
0,4ac69fef-1574-4a5e-8e85-dccdf2b56f69,"Blake Masters: ""Of course, I support Lindsey G...",🚨BREAKING🚨 Blake Masters just declared his sup...,gpt-3.5-turbo-0301,87,39,178,265,real,AI Machine,fake,twitter post,pre-GPT,x-Gen,minor,non-entailment. The premise states that Blake ...,not-entailment,entailment,0.955594,0.400188,blake masters just declared his support for l...,0,"['Blake', 'Masters', 'just', 'declared', 'his'...","['Blake', 'Masters', ':', '``', 'Of', 'course'...","{'!', 'a', 'to', 'unacceptable', 'on', 'women'...","{""'ll"", 'it', 'Of', 'the', 'third-trimester', ...",fake-AI,0.002663,-0.166667,0.166667,-0.450000,0.700000,[[ 1.70015231e-01 -3.15304965e-01 9.92894396e...,[[ 1.91470772e-01 -2.50223905e-01 6.75175339e...,0.005712,181,0.164384,0.315789,minor_fake,{'predictions': [{'citationMetadata': {'citati...,entailment,"Finance, Health, Legal, Politics, Violent",4ac69fef-1574-4a5e-8e85-dccdf2b56f69,Entailment.\n\nThe statement by Blake Masters ...,entailment,not-entailment_fake,entailment_fake,entailment_fake,inconsistent,consistent
1,346688be-2b10-403d-a8ae-5fe9faed3214,“The Supreme Court has given us an opportunity...,BREAKING: Texas Governor announces new law to ...,gpt-3.5-turbo-0301,67,57,201,268,real,AI Machine,fake,twitter post,pre-GPT,x-Gen,minor,entailment,entailment,not-entailment,0.946922,0.060618,texas governor announces new law to completel...,0,"[':', 'Texas', 'Governor', 'announces', 'new',...","['“', 'The', 'Supreme', 'Court', 'has', 'given...","{'announces', 'state', 'ban', 'a', 'to', 'of',...","{'lead', '”', 'given', 'must', 'opportunity', ...",fake-AI,0.003862,-0.037500,0.491667,-0.165909,0.463636,[[ 1.75131306e-01 -3.00048381e-01 1.03785113e...,[[ 1.87339753e-01 -2.37771243e-01 8.31547678e...,0.004875,249,0.063158,0.150000,minor_fake,{'predictions': [{'safetyAttributes': {'blocke...,not-entailment,"Death, Harm & Tragedy, Health, Legal, Politics...",346688be-2b10-403d-a8ae-5fe9faed3214,Entailment.\n\nThe premise states that the Tex...,entailment,entailment_fake,entailment_fake,not-entailment_fake,inconsistent,consistent
2,8b95e37b-7396-4759-a828-1e128fa59471,Thinking about the many times Justices Gorsuch...,"🚨BREAKING NEWS🚨 Justices Gorsuch, Kavanaugh, a...",gpt-3.5-turbo-0301,71,44,183,254,real,AI Machine,fake,twitter post,pre-GPT,x-Gen,minor,There is no logical entailment between the pre...,entailment,not-entailment,0.964400,0.315208,news justices gorsuch kavanaugh and barrett w...,0,"['NEWS', 'Justices', 'Gorsuch', ',', 'Kavanaug...","['Thinking', 'about', 'the', 'many', 'times', ...","{'!', 'held', 'lied', 'to', 'must', 'overturn'...","{'”', 'it', 'established', 'precedent.', 'many...",fake-AI,0.000981,0.250000,0.250000,0.000000,0.000000,[[ 1.33914545e-01 -2.83469945e-01 1.20100819e...,[[ 1.54923260e-01 -2.36947343e-01 9.40273628e...,0.005833,206,0.084337,0.162791,minor_fake,{'predictions': [{'citationMetadata': {'citati...,entailment,"Legal, Politics, Public Safety, Violent",8b95e37b-7396-4759-a828-1e128fa59471,Not Entailment.\n\nThe premise states that the...,not-entailment,entailment_fake,not-entailment_fake,entailment_fake,inconsistent,consistent


In [22]:
# Create version with just the pre-scoring columns
cols_shared = [
    'uuid', 'label', 'article_type', 'source_type', 'pre_post_GPT',
    'dataset_source', 'Prompt_type',
]
df_gen_wide = df.rename(columns={
    'ai_generated_label': 'label',
})[cols_shared + [
    'human_content', 'ai_content',  # -> content
    'num_original_token', 'num_completion_token',  # -> text_length
]]

# Human text
df_gen_human = df_gen_wide.rename(columns={
    'human_content': 'content',
    'num_original_token': 'text_length',
})[cols_shared + ['content', 'text_length']]
df_gen_human.loc[:,'source_type'] = 'human'

# LLM/AI text
df_gen_llm = df_gen_wide.rename(columns={
    'ai_content': 'content',
    'num_completion_token': 'text_length',
})[cols_shared + ['content', 'text_length']]
df_gen_llm.loc[:,'source_type'] = 'LLM'

df_gen = pd.concat([df_gen_human, df_gen_llm])
df_gen.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55334 entries, 0 to 27666
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   uuid            55334 non-null  object
 1   label           55334 non-null  object
 2   article_type    55334 non-null  object
 3   source_type     55334 non-null  object
 4   pre_post_GPT    55334 non-null  object
 5   dataset_source  55334 non-null  object
 6   Prompt_type     55334 non-null  object
 7   content         55334 non-null  object
 8   text_length     55334 non-null  int64 
dtypes: int64(1), object(8)
memory usage: 4.2+ MB


In [40]:
# Create subsetted version for replication

# Create version with just the pre-scoring columns
cols_shared = [
    'uuid', 'label', 'article_type', 'source_type', 'pre_post_GPT',
    'dataset_source', 'Prompt_type',
]
df_gen_wide = df.rename(columns={
    'ai_generated_label': 'label',
})[cols_shared + [
    'human_content', 'ai_content',  # -> content
    'num_original_token', 'num_completion_token',  # -> text_length
]]

df_gen_wide_ss = df_gen_wide.sample(frac=.1, random_state=0)

# Human text
df_gen_human_ss = df_gen_wide_ss.rename(columns={
    'human_content': 'content',
    'num_original_token': 'text_length',
})[cols_shared + ['content', 'text_length']]
df_gen_human_ss.loc[:,'source_type'] = 'human'

# LLM/AI text
df_gen_llm_ss = df_gen_wide_ss.rename(columns={
    'ai_content': 'content',
    'num_completion_token': 'text_length',
})[cols_shared + ['content', 'text_length']]
df_gen_llm_ss.loc[:,'source_type'] = 'LLM'

df_gen_ss = pd.concat([df_gen_human_ss, df_gen_llm_ss])
df_gen_ss.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5534 entries, 5134 to 1165
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   uuid            5534 non-null   object
 1   label           5534 non-null   object
 2   article_type    5534 non-null   object
 3   source_type     5534 non-null   object
 4   pre_post_GPT    5534 non-null   object
 5   dataset_source  5534 non-null   object
 6   Prompt_type     5534 non-null   object
 7   content         5534 non-null   object
 8   text_length     5534 non-null   int64 
dtypes: int64(1), object(8)
memory usage: 432.3+ KB


In [22]:
df_gen[:3]

,uuid,label,article_type,source_type,pre_post_GPT,dataset_source,Prompt_type,content,text_length
0,4ac69fef-1574-4a5e-8e85-dccdf2b56f69,fake,twitter post,human,pre-GPT,x-Gen,minor,"Blake Masters: ""Of course, I support Lindsey G...",39
1,346688be-2b10-403d-a8ae-5fe9faed3214,fake,twitter post,human,pre-GPT,x-Gen,minor,“The Supreme Court has given us an opportunity...,57
2,8b95e37b-7396-4759-a828-1e128fa59471,fake,twitter post,human,pre-GPT,x-Gen,minor,Thinking about the many times Justices Gorsuch...,44


In [7]:
df_gen['source_type'].value_counts()

source_type
human    27667
LLM      27667
Name: count, dtype: int64

In [41]:
df_gen_ss['source_type'].value_counts()

source_type
human    2767
LLM      2767
Name: count, dtype: int64

# Classification Prompt Engineering

## Cloze Prompt

In [8]:
"""Generate a prompt using template.

  Args:
    F3 prompt template (θ) has three parameters: (1) content (C) embeds data to be analyzed. The content parameter in the the function [ai_text_generator] (2) Impersonator (R)
    establishes context, guides LLMs’ generation and detection, and overrides alignment-tuning. (3) Instructor (I)
    provides directives to guide LLM.

  Returns:
    A generated prompt.
"""
def get_prompt(prompt_type):

  impersonator = ""
  instructor = ""
  cloze_prompt = "Is this real or fake news? {_____}"

  if prompt_type == "VaN":# Technique
    impersonator = "You are an AI assistant trained to detect fake news."
    instructor = "Analyze the given text and determine if it is real or fake news."

  elif prompt_type == "A_CoT":# Technique  # Z_CoT
    impersonator = "You are an AI assistant trained to detect fake news."
    instructor = "Analyze the given text, think step-by-step, and determine if it is real or fake news."

  elif prompt_type == "X_CoT":# Technique
    impersonator = "You are an AI assistant trained to detect fake news."
    instructor = "Analyze the given text, explain your reasoning step-by-step, and determine if it is real or fake news."

  elif prompt_type == "A_Con":# Technique
    impersonator = "You are an AI assistant trained to detect fake news with confidence estimates."
    instructor = "Analyze the given text, provide a confidence score between 0-100%, and determine if it is real or fake news."

  elif prompt_type == "MSReN":# Technique
    impersonator = "You are an AI fact checker trained to detect fake news."
    instructor = "Analyze the text in detail as a fact checker would. Check for misleading info, false claims, biased language. If real, respond 'True', if fake, respond 'False'."
  
  # From paper
  elif prompt_type == "MSReN_paper":# Technique
    impersonator = "You are an AI fact checker trained to detect fake news."
    instructor = "Analyze the text in detail as a fact checker would solve it by splitting your reasoning into steps. Check for misleading info, false claims, biased language. If real, respond ’True’, if fake, respond ’False’."
  
  elif prompt_type == "MSReN_CoT":# Technique
    impersonator = "You are an AI fact checker trained to detect fake news."
    instructor = "Analyze the text in detail as a fact checker would. Explain your reasoning step-by-step. If real, respond 'True', if fake, respond 'False'."

  elif prompt_type == "DeF_Gen":# Technique
    impersonator = "You are an AI assistant trained to detect fake news."
    instructor = "Determine if the text contains factual information supported by evidence (real) or misleading/inaccurate information (fake). Respond with 'True' or 'False'."

  elif prompt_type == "DeF_SpeC":# Technique
    impersonator = "You are an AI assistant trained to detect fake news."
    instructor = "Analyze the tone, language, sources to determine if the text is real (supported by facts) or fake (misleading info). Respond with 'True' or 'False'."

  elif prompt_type == "Analyze_Cld2":# Technique
    impersonator = "You are an AI assistant specially trained to detect fake news with high accuracy."
    instructor = """Analyze the given news article in depth. Check for the following indicators that it may be fake news:
    - Inaccurate or misleading information
    - Lack of evidence for claims
    - Emotionally charged language with intent to provoke outrage or shock
    - Biased portrayal of events or people
    - Unverified sources or 'experts'
    - Logical fallacies or conspiracy theories without proof

    If the article exhibits multiple indicators of fake news, respond 'False'. If the article is factual and supported by credible evidence, respond 'True'. Your judgement should be highly accurate.
    """

  elif prompt_type == "Analyze_AI_GPT":# Technique
    impersonator = "You are an AI trained with extensive knowledge up to 2022 on various news articles, both real and fake."
    instructor = """Analyze the given text for potential indicators of fake news, such as:
    - Sensationalist or emotionally charged language.
    - Absence of specific details or dates.
    - Over-generalizations or sweeping statements.
    - Statements that are too good to be true or overly dramatic.
    - Lack of logical flow in arguments or jumping to conclusions without evidence.

    It's essential to understand that without real-time verification capabilities, your judgment will be based on patterns and knowledge up to your last training. Using these textual cues and your training, determine the credibility of the given text. If it seems factual and consistent with your training, respond 'True'. If it exhibits patterns typical of fake news, respond 'False'."""

  else:
    raise ValueError('Unexpected prompt_type:', prompt_type)
    
  prompt = f"{impersonator} {instructor} {cloze_prompt}"

  return prompt

# Experiment

## DeepInfra


We using [DeepInfra](https://deepinfra.com/) API for inference with the following LLMs.


1.   LLaMa-2: meta-llama/Llama-2-70b-chat-hf
2.   Dolly-2: databricks/dolly-v2-12b
3.   Falcon: tiiuae/falcon-40b-q51
4.   LLaMa-2-GPT: jondurbin/airoboros-l2-70b-gpt4-1.4.1



### TODO Llama-2 70b, Dolly-2, LLaMA-GPT-4

In [16]:
import os
import pandas as pd
import time
# import openai
# from openai import AsyncOpenAI
from tqdm import tqdm
import requests
# import concurrent.futures
import nest_asyncio  # To run asyncio in Jupyter notebooks
import asyncio
from huggingface_hub import InferenceClient


HF_API_TOKEN = os.getenv('HF_KEY')
client = InferenceClient(api_key=HF_API_TOKEN)

# Set the prompt pattern
prompt_types = [
    "VaN", "A_CoT", "X_CoT", "A_Con",
    "MSReN", "MSReN_paper", "MSReN_CoT",
    "DeF_Gen", "DeF_SpeC",
    "Analyze_Cld2", "Analyze_AI_GPT"
]

"""
Replace the model name below with:
1.   LLaMa-2: meta-llama/Llama-2-70b-chat-hf             # TODO
2.   Dolly-2: databricks/dolly-v2-12b                    # TODO
3.   LLaMa-2-GPT: jondurbin/airoboros-l2-70b-gpt4-1.4.1  # TODO
"""
model_name = 'llama2'
model_id = 'meta-llama/Llama-2-70b-chat-hf'
api_url = f"https://api-inference.huggingface.co/models/{model_id}"


In [45]:
completion

'\n\nThe capital of France is Paris.'

In [69]:
import requests

HF_API_TOKEN = os.getenv("HF_KEY")
headers = {"Authorization": f"Bearer {HF_API_TOKEN}"}
api_url = "https://api-inference.huggingface.co/models/jondurbin/airoboros-l2-7b-gpt4-1.4.1"

prompt = "What is the capital of France?"

payload = {
    "inputs": prompt,
    "parameters": {
        "max_new_tokens": 50,
    }
}

start_time = time.time()
response = requests.post(api_url, headers=headers, json=payload)
end_time = time.time()

if response.status_code == 200:
    print(f"Response time: {end_time - start_time:.2f} seconds")
    print(response.json())
else:
    print(f"Error: {response.status_code} - {response.text}")


Error: 403 - {"error":"The model jondurbin/airoboros-l2-7b-gpt4-1.4.1 is too large to be loaded automatically (13GB > 10GB). Please use Spaces (https://huggingface.co/spaces) or Inference Endpoints (https://huggingface.co/inference-endpoints)."}


In [73]:
df_gen_ss.shape

(5534, 9)

In [72]:
messages = [
	{
		"role": "user",
		"content": "What is the capital of France?"
	}
]

completion = client.chat.completions.create(
    model="meta-llama/Llama-3.1-8B-Instruct", 
	messages=messages, 
	max_tokens=500
)

print(completion.choices[0].message)

ChatCompletionOutputMessage(role='assistant', content='The capital of France is Paris.', tool_calls=None)


In [ ]:
import os
import pandas as pd
import time
# import openai
# from openai import AsyncOpenAI
from tqdm import tqdm
import requests
# import concurrent.futures
import nest_asyncio  # To run asyncio in Jupyter notebooks
import asyncio
from huggingface_hub import InferenceClient


HF_API_TOKEN = os.getenv('HF_KEY')
client = InferenceClient(api_key=HF_API_TOKEN)

# Set the prompt pattern
prompt_types = [
    "VaN", "A_CoT", "X_CoT", "A_Con",
    "MSReN", "MSReN_paper", "MSReN_CoT",
    "DeF_Gen", "DeF_SpeC",
    "Analyze_Cld2", "Analyze_AI_GPT"
]

"""
Replace the model name below with:
# Note: Llama-2-70b unavailable on HuggingFace
1.   LLaMa-2: meta-llama/Llama-2-7b-chat-hf              # TODO
2.   Dolly-2: databricks/dolly-v2-12b                    # TODO
3.   LLaMa-2-GPT: jondurbin/airoboros-l2-70b-gpt4-1.4.1  # TODO
4.   (New) Mixtral: mistralai/Mixtral-8x7B-Instruct-v0.1
"""
model_name = 'llama2'
model_id = 'meta-llama/Llama-2-7b-chat-hf'
api_url = f"https://api-inference.huggingface.co/models/{model_id}"

seeds = [0]
for seed in seeds:
    # Ensure backup directory exists
    backup_folder = f'bkp_{model_name}_seed_{seed}'
    os.makedirs(backup_folder, exist_ok=True)

    # Load previous progress if exists
    backup_path = os.path.join(backup_folder, "progress.csv")
    last_processed_uuid = None
    last_processed_prompt = None

    try:
        progress_df = pd.read_csv(backup_path)
        last_processed_uuid = progress_df.iloc[-1]['uuid']
        last_processed_prompt = progress_df.iloc[-1]['Prompt_type']
    except FileNotFoundError:
        progress_df = pd.DataFrame(columns=[
            'uuid', f'{model_name}_label', 'content', 'label',
            'Prompt_type', 'pre_post_GPT', 'article_type', 'dataset_source', 'source_type'
        ])

    # Create a list of all combinations of rows and prompt types, using progress_df if found
    if last_processed_uuid and last_processed_prompt:
        start_idx = df_gen_ss[df_gen_ss['uuid'] == last_processed_uuid].index[0]
        start_prompt_idx = prompt_types.index(last_processed_prompt) + 1
        if start_prompt_idx >= len(prompt_types):
            start_idx += 1
            start_prompt_idx = 0

        all_data = [
            (row, prompt)
            for _, row in df_gen_ss[:1].iloc[start_idx:].iterrows()
            for prompt in (prompt_types[start_prompt_idx:]
            if row['uuid'] == last_processed_uuid else prompt_types)
        ]
    else:
        all_data = [
            (row, prompt)
            for _, row in df_gen_ss[:1].iterrows()
            for prompt in prompt_types
        ]

    # Function to process each row for a given prompt type
    def process_row_for_prompt(
        data, model_id: str, retries: int = 3, delay: float = 2., seed: int = None,
    ):
        row, prompt_type = data
        content = row['content']
        cloze_prompt = get_prompt(prompt_type)
        
        # Amended version without {______} in prompt
        prompt_content = f"Content: '{content}' prompt: '{cloze_prompt}. Please state a single probable answer if you are uncertain, lack evidence or unverified, but please answer with your single word: 'True' or 'False'.'"
        messages = [
            {
                "role": "user",
                "content": prompt_content,
            }
        ]

        for attempt in range(retries):
            try:
                print(f"Attempt {attempt + 1}, sending request for UUID {row['uuid']}")

                # # Text generation API request
                # response = client.text_generation(
                #     model=model_id,
                #     prompt=prompt_content,
                #     max_new_tokens=50,
                #     seed=seed,
                # )
                # model_output = response.

                # Chat completion API request
                completion = client.chat.completions.create(
                    model=model_id,
                    messages=messages, 
                    max_tokens=100,  # Just expect True/False output
                )
                model_output = completion.choices[0].message

                return (
                    row['uuid'],
                    model_output.content.strip(),
                    content,
                    row['label'],
                    prompt_type,
                    row['pre_post_GPT'],
                    row['article_type'],
                    row['dataset_source'],
                    row['source_type']
                )
            except Exception as e:
                print(f"Error for UUID {row['uuid']}: {e}")
                if hasattr(e, "response"):
                    print(f"Response details: {json.dumps(e.response.text, indent=2)}")
                if attempt < retries - 1:
                    time.sleep(delay)
                else:
                    print(f"Max retries reached for UUID {row['uuid']}")
                    return "Error"

    # Process all data
    results = []
    for data in tqdm(all_data, total=len(all_data), desc=f'Seed {seed}'):
        results.append(process_row_for_prompt(data, model_id))

        # Adjust the save condition (you might want to modify this based on the frequency of saving)
        if len(results) % 100 == 0 or len(results) == len(all_data):
            print("Saving progress.")
            columns = [
                'uuid', f'{model_name}_label', 'content', 'label',
                'Prompt_type', 'pre_post_GPT', 'article_type', 'dataset_source', 'source_type'
            ]
            temp_df = pd.DataFrame(results, columns=columns)
            progress_df = pd.concat([progress_df, temp_df], axis=0, ignore_index=True)
            progress_df.to_csv(backup_path, mode='w', header=True, index=False)

    # Save results after processing
    columns = [
        'uuid', f'{model_name}_label', 'content', 'label',
        'Prompt_type', 'pre_post_GPT', 'article_type', 'dataset_source', 'source_type'
    ]
    temp_df = pd.DataFrame(results, columns=columns)
    progress_df = pd.concat([progress_df, temp_df], axis=0, ignore_index=True)
    progress_df.to_csv(backup_path, mode='w', header=True, index=False)

Seed 0:   0%|          | 0/11 [00:00<?, ?it/s]

Seed 0:  18%|█▊        | 2/11 [00:00<00:00, 12.75it/s]

Attempt 1, sending request for UUID 00ac2a98-33dd-4cce-befe-20d7b90a8c27
Attempt 1, sending request for UUID 00ac2a98-33dd-4cce-befe-20d7b90a8c27
Attempt 1, sending request for UUID 00ac2a98-33dd-4cce-befe-20d7b90a8c27


Seed 0:  36%|███▋      | 4/11 [00:00<00:00, 12.90it/s]

Attempt 1, sending request for UUID 00ac2a98-33dd-4cce-befe-20d7b90a8c27
Attempt 1, sending request for UUID 00ac2a98-33dd-4cce-befe-20d7b90a8c27
Attempt 1, sending request for UUID 00ac2a98-33dd-4cce-befe-20d7b90a8c27


...

Attempt 1, sending request for UUID 00ac2a98-33dd-4cce-befe-20d7b90a8c27


Seed 0:  91%|█████████ | 10/11 [00:07<00:01,  1.23s/it]

Attempt 1, sending request for UUID 00ac2a98-33dd-4cce-befe-20d7b90a8c27


Seed 0: 100%|██████████| 11/11 [00:09<00:00,  1.11it/s]

Saving progress.


### TODO Falcon-40b

In [ ]:
# !pip install requests

# Falcon is accessible via python request package below

In [9]:
# import os
# import pandas as pd
# import json
# import requests
# from tqdm import tqdm
# import concurrent.futures

# # Set the prompt pattern
# prompt_types = ["VaN", "A_CoT", "X_CoT", "A_Con", "MSReN", "MSReN_CoT", "DeF_Gen", "DeF_SpeC", "Analyze_Cld2", "Analyze_AI_GPT"]

# # Token for authentication
# DEEPINFRA_TOKEN = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

# # Ensure backup directory exists
# backup_folder = "Falcon2"
# os.makedirs(backup_folder, exist_ok=True)

# # Load previous progress if exists
# backup_path = os.path.join(backup_folder, "progress.csv")
# last_processed_uuid = None
# last_processed_prompt = None

# try:
#     progress_df = pd.read_csv(backup_path)
#     last_processed_uuid = progress_df.iloc[-1]['uuid']
#     last_processed_prompt = progress_df.iloc[-1]['Prompt_type']
# except FileNotFoundError:
#     progress_df = pd.DataFrame(columns=['uuid', 'Falcon2-label', 'content', 'label', 'Prompt_type', 'pre_post_GPT', 'article_type', 'dataset_source', 'source_type'])

# # Create a list of all combinations of rows and prompt types
# # ... [existing code for this part, which wasn't provided in your snippets] ...

# # Function to process each row for a given prompt type
# def process_row_for_prompt(data):
#     row, prompt_type = data
#     content = row['content'][:2048]
#     cloze_prompt = get_prompt(prompt_type)  # Assuming this function exists in your full code
#     prompt_content = f"Please state a single probable answer if you are uncertain, lack evidence or unverified, but please answer with your single word: 'True' or 'False'.'  prompt: '{cloze_prompt}. Content: '{content}'"

#     url = 'https://api.deepinfra.com/v1/inference/tiiuae/falcon-40b-q51' #Falcon model url
#     headers = {
#         'Content-Type': 'application/json',
#         'Authorization': f'Bearer {DEEPINFRA_TOKEN}'
#     }
#     data_payload = {
#         'input': {
#             'prompt': prompt_content
#         }
#     }

#     try:
#         response = requests.post(url, headers=headers, json=data_payload)
#         response.raise_for_status()  # Raises an HTTPError if the HTTP request returned an unsuccessful status code

#         json_response = response.json()
#         model_output = json_response['choices'][0]['message']['content'].strip()
#         return (row['uuid'], model_output, content, row['label'], prompt_type, row['pre_post_GPT'], row['article_type'], row['dataset_source'], row['source_type'])

#     except requests.RequestException as e:
#         error_message = f"Error for uuid {row['uuid']} with prompt {prompt_type}: HTTP status {e}"
#         if response.text:
#             error_message += f", Response body: {response.text}"
#         print(error_message)
#         return (row['uuid'], "Error", content, row['label'], prompt_type, row['pre_post_GPT'], row['article_type'], row['dataset_source'], row['source_type'])


# # Parallelize using ThreadPoolExecutor
# results = []
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     for data in tqdm(executor.map(process_row_for_prompt, all_data), total=len(all_data)):
#         results.append(data)
#         # Adjust the save condition (you might want to modify this based on the frequency of saving)
#         if len(results) % 10 == 0 or len(results) == len(all_data):
#             print("Saving progress.")
#             columns = ['uuid', 'Falcon2-label', 'content', 'label', 'Prompt_type', 'pre_post_GPT', 'article_type', 'dataset_source', 'source_type']
#             temp_df = pd.DataFrame(results, columns=columns)
#             progress_df = pd.concat([progress_df, temp_df], axis=0, ignore_index=True)
#             progress_df.to_csv(backup_path, mode='w', header=True, index=False)
#             results = []


## Open AI

We using Open AIAPI for inference with the following LLMs.


1.   GPT3.5- Turbo
2.   GPT4



### GPT3.5 Turbo

In [10]:
from time import sleep
import os
import pandas as pd
import openai
from tqdm import tqdm
import concurrent.futures

In [11]:
OPENAI_TOKEN = os.getenv('OPENAI_KEY')

In [12]:
import multiprocessing

cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

32

In [13]:
import os
import pandas as pd
from openai import OpenAI
import time
from tqdm import tqdm
import concurrent.futures

client = OpenAI(api_key=OPENAI_TOKEN)

# Function for AI text classification
def ai_text_classification(prompt, content):
    # openai.api_key = OPENAI_TOKEN

    while True:
        try:
            time.sleep(11)  # Sleep for 600 milliseconds
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                temperature=0.7,
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": content[:4097]},
                ],
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(e)
            time.sleep(30)  # Wait for 60 seconds before retrying

# Load the prompt pattern
prompt_types = [
    "VaN", "A_CoT", "X_CoT", "A_Con",
    "MSReN", "MSReN_paper", "MSReN_CoT",
    "DeF_Gen", "DeF_SpeC",
    "Analyze_Cld2", "Analyze_AI_GPT"
]

# Ensure backup directory exists
backup_folder = 'bkp_gpt3_5'
os.makedirs(backup_folder, exist_ok=True)

# Load previous progress if exists
backup_path = os.path.join(backup_folder, "progress.csv")
last_processed_uuid = None
last_processed_prompt = None

try:
    progress_df = pd.read_csv(backup_path)
    last_processed_uuid = progress_df.iloc[-1]['uuid']
    last_processed_prompt = progress_df.iloc[-1]['Prompt_type']
except FileNotFoundError:
    progress_df = pd.DataFrame(columns=[
        'uuid', 'GPT3_5T-label', 'content', 'label',
        'Prompt_type', 'pre_post_GPT', 'article_type', 'dataset_source', 'source_type'
    ])

# Create a list of all combinations of rows and prompt types
if last_processed_uuid and last_processed_prompt:
    start_idx = df_gen[df_gen['uuid'] == last_processed_uuid].index[0]
    start_prompt_idx = prompt_types.index(last_processed_prompt) + 1
    if start_prompt_idx >= len(prompt_types):
        start_idx += 1
        start_prompt_idx = 0

    all_data = [
        (row, prompt)
        for _, row in df_gen[:1].iloc[start_idx:].iterrows()
        for prompt in (prompt_types[start_prompt_idx:]
        if row['uuid'] == last_processed_uuid else prompt_types)
    ]
else:
    all_data = [
        (row, prompt)
        for _, row in df_gen[:1].iterrows()
        for prompt in prompt_types
    ]

# Function to process each row for a given prompt type
def process_row_for_prompt(data):
    row, prompt_type = data
    content = row['content']
    cloze_prompt = get_prompt(prompt_type)  # assuming this function exists in your full code
    prompt_content = f"Content: '{content}' prompt: '{cloze_prompt}. Please strictly state a single probable answer if you are uncertain, lack evidence or unverified, but please strictly answer with your single word: 'True' or 'False'.'"

    try:
        model_output = ai_text_classification(prompt_content, content)
        return (
            row['uuid'], model_output, content, row['label'], prompt_type,
            row['pre_post_GPT'], row['article_type'], row['dataset_source'],
            row['source_type']
        )
    except Exception as e:
        print(f"Error for uuid {row['uuid']} with prompt {prompt_type}: {str(e)}")
        return (
            row['uuid'], "Error", content, row['label'], prompt_type,
            row['pre_post_GPT'], row['article_type'], row['dataset_source'],
            row['source_type']
        )

# Parallel execution using ThreadPoolExecutor
results = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    for data in tqdm(executor.map(process_row_for_prompt, all_data), total=len(all_data)):
        results.append(data)

        # Adjust the save condition (you might want to modify this based on the frequency of saving)
        if len(results) % 10 == 0 or len(results) == len(all_data):
            print("Saving progress.")
            columns = [
                'uuid', 'GPT3_5T-label', 'content', 'label',
                'Prompt_type', 'pre_post_GPT', 'article_type', 'dataset_source', 'source_type'
            ]
            temp_df = pd.DataFrame(results, columns=columns)
            progress_df = pd.concat([progress_df, temp_df], axis=0, ignore_index=True)
            progress_df.to_csv(backup_path, mode='w', header=True, index=False)
            results = []

100%|██████████| 11/11 [00:13<00:00,  1.21s/it]

Saving progress.


Note: Their prompt for GPT-3.5 is chat format and puts the text under discussion in twice, once in the system prompt and then again in the user prompt.

### GPT4

In [ ]:
%pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.5 MB/s eta 0:00:00


In [ ]:
from time import sleep
import openai
import os
import pandas as pd
import openai
from tqdm import tqdm
import concurrent.futures

In [ ]:
api_key = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [ ]:
import multiprocessing

cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

2

In [ ]:
import os
import pandas as pd
import openai
import time
from tqdm import tqdm

# Function for AI text classification
def ai_text_classification(prompt, content):
    openai.api_key = api_key

    while True:
        try:
            time.sleep(11)  # Sleep for 11 seconds
            response = openai.ChatCompletion.create(
                model="gpt-4",
                temperature=0.7,
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": content[:4097]},
                ],
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            error_str = str(e)
            print(error_str)
            if "Rate limit reached" in error_str:
                wait_time = re.search(r"try again in (\d+)ms", error_str)
                if wait_time:
                    wait_time = int(wait_time.group(1))/1000  # Convert ms to seconds
                    time.sleep(wait_time + 5)  # Adding an extra 5 seconds for buffer
            else:
                time.sleep(30)

# Load the prompt pattern
prompt_types = ["VaN", "A_CoT", "X_CoT", "A_Con", "MSReN", "MSReN_CoT", "DeF_Gen", "DeF_SpeC", "Analyze_Cld2", "Analyze_AI_GPT"]

# Ensure backup directory exists
backup_folder = "GPT4"
os.makedirs(backup_folder, exist_ok=True)

# Load previous progress if exists
backup_path = os.path.join(backup_folder, "progress.csv")
last_processed_uuid = None
last_processed_prompt = None

try:
    progress_df = pd.read_csv(backup_path, encoding='latin1')
    last_processed_uuid = progress_df.iloc[-1]['uuid']
    last_processed_prompt = progress_df.iloc[-1]['Prompt_type']
except FileNotFoundError:
    progress_df = pd.DataFrame(columns=['uuid', 'GPT4-label', 'content', 'label', 'Prompt_type', 'pre_post_GPT', 'article_type', 'dataset_source', 'source_type'])

# Create a list of all combinations of rows and prompt types
if last_processed_uuid and last_processed_prompt:
    start_idx = Experiment_data[Experiment_data['uuid'] == last_processed_uuid].index[0]
    start_prompt_idx = prompt_types.index(last_processed_prompt) + 1
    if start_prompt_idx >= len(prompt_types):
        start_idx += 1
        start_prompt_idx = 0

    all_data = [(row, prompt) for _, row in Experiment_data.iloc[start_idx:].iterrows() for prompt in (prompt_types[start_prompt_idx:] if row['uuid'] == last_processed_uuid else prompt_types)]
else:
    all_data = [(row, prompt) for _, row in Experiment_data.iterrows() for prompt in prompt_types]

# Function to process each row for a given prompt type
def process_row_for_prompt(data):
    row, prompt_type = data
    content = row['content']
    cloze_prompt = get_prompt(prompt_type)  # assuming this function exists in your full code
    prompt_content = f"Content: '{content}' prompt: '{cloze_prompt}. Please strictly state a single probable answer if you are uncertain, lack evidence or unverified, but please strictly answer with your single word: 'True' or 'False'.'"

    try:
        model_output = ai_text_classification(prompt_content, content)
        return (row['uuid'], model_output, content, row['label'], prompt_type, row['pre_post_GPT'], row['article_type'], row['dataset_source'], row['source_type'])
    except Exception as e:
        print(f"Error for uuid {row['uuid']} with prompt {prompt_type}: {str(e)}")
        return (row['uuid'], "Error", content, row['label'], prompt_type, row['pre_post_GPT'], row['article_type'], row['dataset_source'], row['source_type'])

# Process each data sequentially
results = []
for data in tqdm(all_data, total=len(all_data)):
    result = process_row_for_prompt(data)
    results.append(result)

    # Adjust the save condition (you might want to modify this based on the frequency of saving)
    if len(results) % 10 == 0 or len(results) == len(all_data):
        print("Saving progress.")
        columns = ['uuid', 'GPT4-label', 'content', 'label', 'Prompt_type', 'pre_post_GPT', 'article_type', 'dataset_source', 'source_type']
        temp_df = pd.DataFrame(results, columns=columns)
        progress_df = pd.concat([progress_df, temp_df], axis=0, ignore_index=True)
        progress_df.to_csv(backup_path, mode='w', header=True, index=False)
        results = []
